In [47]:
import numpy as np
import pandas as pd
import heapq

In [241]:
n_steps = 10
beam_width = 6
n_expand = 20

In [242]:
def add_noise(x):
    return np.round(x+np.random.random()*10,0)
def repeat_sample(n_expand,seq,sh):
    rows = []
    for _ in range(n_expand):
        rows.append({'seq':seq+[int(add_noise(0))],'r':add_noise(sh)})
    seq_df = pd.DataFrame(rows)
    return seq_df

In [243]:
big_Q = np.ones(n_steps+1)*-np.infty
d = {}
rew_0 = 0
init = 0 #initial action in sequence
init_heap = [(rew_0,[init])]
big_Q[init] = rew_0
d[init] = init_heap #initialize the heap
POPS = {}
while n_steps not in d.keys(): #as long as the beams have not reached the n_steps length
    queue_sort = np.argsort(big_Q)
    t = queue_sort[-1]
    sh,seq = d[t][-1]
    d[t] = d[t][0:-1]
    d[t].sort()
    if len(d[t]) == 0:
        big_Q[t] = -np.infty
    else:
        best_remaining_sh = d[t][-1][0]
        big_Q[t] = best_remaining_sh

    if t not in POPS:
        POPS[t] = 0

    if POPS[t] >= beam_width or t > n_steps:
        continue
    POPS[t] += 1

    seq_df = repeat_sample(n_expand,seq,sh)
    for idx,row in seq_df.iterrows():
        seq_i,rew_i = row['seq'],row['r']
        
    #     #Check if t+1 is in d
        if t+1 in d.keys():
            if len(d[t+1]) < beam_width:
                d[t+1].append((rew_i,seq_i))
                d[t+1].sort()
            else: #We have to check for dominated sequences
                if d[t+1][0][0] < rew_i: #We replace them
                    d[t+1][0] =  (rew_i,seq_i)
                    d[t+1].sort()
        else: #we need toq initialize B_t+1
            init_heap = [(rew_i,seq_i)]
            d[t+1] = init_heap
    best_sh_t1 = d[t+1][-1][0]
    big_Q[t+1] = best_sh_t1
big_Q,d
    

(array([-inf,   9.,  18.,  28.,  38.,  46.,  56.,  65.,  74.,  84.,  93.]),
 {0: [],
  1: [(8.0, [0, 2]),
   (8.0, [0, 7]),
   (8.0, [0, 9]),
   (9.0, [0, 5]),
   (9.0, [0, 7])],
  2: [(18.0, [0, 8, 0]),
   (18.0, [0, 8, 1]),
   (18.0, [0, 8, 1]),
   (18.0, [0, 8, 1]),
   (18.0, [0, 8, 10])],
  3: [(26.0, [0, 8, 6, 9]),
   (27.0, [0, 8, 6, 4]),
   (27.0, [0, 8, 6, 6]),
   (28.0, [0, 8, 6, 5]),
   (28.0, [0, 8, 6, 7])],
  4: [(37.0, [0, 8, 6, 6, 6]),
   (37.0, [0, 8, 6, 6, 6]),
   (37.0, [0, 8, 6, 6, 8]),
   (38.0, [0, 8, 6, 6, 4]),
   (38.0, [0, 8, 6, 6, 5])],
  5: [(45.0, [0, 8, 6, 6, 9, 9]),
   (46.0, [0, 8, 6, 6, 9, 1]),
   (46.0, [0, 8, 6, 6, 9, 3]),
   (46.0, [0, 8, 6, 6, 9, 7]),
   (46.0, [0, 8, 6, 6, 9, 9])],
  6: [(54.0, [0, 8, 6, 6, 9, 8, 3]),
   (55.0, [0, 8, 6, 6, 9, 8, 5]),
   (56.0, [0, 8, 6, 6, 9, 8, 1]),
   (56.0, [0, 8, 6, 6, 9, 8, 4]),
   (56.0, [0, 8, 6, 6, 9, 8, 8])],
  7: [(62.0, [0, 8, 6, 6, 9, 8, 10, 10]),
   (64.0, [0, 8, 6, 6, 9, 8, 10, 3]),
   (64.0, [0, 8, 6, 